In [1]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent, AgentState
from langchain.messages import HumanMessage, ToolMessage
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

In [2]:
load_dotenv()

True

In [3]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")

In [4]:
class CustomState(AgentState):
    favourite_colour : str

# Write to state

In [5]:
@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour,
        "messages": [ToolMessage("Successfully updated favourite colour", 
                                 tool_call_id = runtime.tool_call_id)]
    })

In [6]:
agent = create_agent(
    model = model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [7]:
response = agent.invoke(
    {"messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "4"}}
)

In [8]:
from pprint import pp
pp(response)

{'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='1fbdf0e6-dca8-4fc9-b3da-b471e2cca735'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}, '__gemini_function_call_thought_signatures__': {'e57c933b-64fc-480f-8c00-02a075d08304': 'CvoBAXLI2nxsyP0777aRnPuI33vXcjRDcKxjgbEbrIv/PQyaiqRTlUB0xrUAL5Cr/d4DWN3VTZbsBk3P02sieP6S+1hmvL0DdP9zv84jijmi8DlJa048qvnoNZxQULWE8IspuzJXSYWXv0rAHm9ce2wG7RPwRsPXKvlD0l0SzvcdCR0lSpUZWQp8HraUMu7wWF9u4ZuOM8UXi54OlY8eaiONNS+Mm5JvrXlM+mxPIYFtEBx5Qx6QlxitSZCXH61xhcGeo9ND1LV2d6trk/Il9qDYlUmnuf66/Zi/j5kzz9VUXQSErwyWpYhb4qPua0uApq+Ibsp5XyCLq2RPkw=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bca6b-d8ce-7411-8755-a3cc67b2088e-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_

In [11]:
response = agent.invoke(
    {
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id":"10"}}
)

In [12]:
pp(response)

{'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='54d3267d-1e8b-4844-9e2c-5b8a5b1ef23d'),
              AIMessage(content="I'm doing well, thank you! How can I help you today?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bca6e-02d7-7a51-bb8d-82df4748b62b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 65, 'output_tokens': 16, 'total_tokens': 81, 'input_token_details': {'cache_read': 0}}),
              HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='f2032455-73d2-4975-8ec4-ca68faeeb16d'),
              AIMessage(content="I'm doing great, thank you! How can I assist you today?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id=

# Read state

In [13]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read nthe favourite colour of the user from the state."""

    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite color found in state"

In [14]:
agent = create_agent(
    model = model,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [ ]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)



NameError: name 'pprint' is not defined

In [16]:
pp(response)

{'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='0ebd869f-284d-4bb0-be9b-b798fbe1974c'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}, '__gemini_function_call_thought_signatures__': {'4196777d-60e7-411b-be25-ee07aa697471': 'CrcCAXLI2ny7Z8RH3fpgNPAFSKyZvFnvqu0o7Qmj3vMu9Nnmc7pj0U9qUvJZ+1S/k3ajZ8Pt74xeUgR4IMRYb4wA64e7/SktfjyPvpY1Z7/odCztiH/D4FX9QvbmwjqMDyPQI84X8EtvUH4DqG43HIQlWdgrG4mtu30dNSz81PVKZmfnDmpSShiq4RrH4ciDKkjIQqnE2Wqj/1oP8efaaAv0E83/TAe++yM1JFqrpqWzg5eHn0OnS4Tb49I4eN/NoNA0uM3dpNZ4mVam5MoKqL+mELfQKb27TNUwfA9B1raLNPurxx2TZZnD8TCve5mo1PA3YogeB+IkDcxojjkIBpUGvbt0QwiIfihY+xiOHbnya5G/PkyxuNHK2owEWKVo9qzKk/V1yq97M4LXMLocMxJ8zKlko34KqMs='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bca71-5427-7fc0-a98d-93dc5

In [17]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

NameError: name 'pprint' is not defined

In [18]:
pp(response)

{'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='0ebd869f-284d-4bb0-be9b-b798fbe1974c'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}, '__gemini_function_call_thought_signatures__': {'4196777d-60e7-411b-be25-ee07aa697471': 'CrcCAXLI2ny7Z8RH3fpgNPAFSKyZvFnvqu0o7Qmj3vMu9Nnmc7pj0U9qUvJZ+1S/k3ajZ8Pt74xeUgR4IMRYb4wA64e7/SktfjyPvpY1Z7/odCztiH/D4FX9QvbmwjqMDyPQI84X8EtvUH4DqG43HIQlWdgrG4mtu30dNSz81PVKZmfnDmpSShiq4RrH4ciDKkjIQqnE2Wqj/1oP8efaaAv0E83/TAe++yM1JFqrpqWzg5eHn0OnS4Tb49I4eN/NoNA0uM3dpNZ4mVam5MoKqL+mELfQKb27TNUwfA9B1raLNPurxx2TZZnD8TCve5mo1PA3YogeB+IkDcxojjkIBpUGvbt0QwiIfihY+xiOHbnya5G/PkyxuNHK2owEWKVo9qzKk/V1yq97M4LXMLocMxJ8zKlko34KqMs='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bca71-5427-7fc0-a98d-93dc5